In [ ]:
import re
import time
import pandas as pd
from splinter import Browser
from config import aLinkedIn
from Models.linkedin import LinkedIn
from Main import daugherty, slalom, _1904labs, worldWideTechnology

In [ ]:
urls = [daugherty.linkedin, slalom.linkedin, _1904labs.linkedin, worldWideTechnology.linkedin]

In [ ]:
def login():
    browser.visit(urls[0])
    button = browser.find_by_css("a[class='form-toggle']")
    button.click()
    browser.fill('session_key', aLinkedIn["email"])
    browser.fill('session_password', aLinkedIn["password"])
    button = browser.find_by_css("input[class='login submit-button']")
    button.click()
    

In [ ]:
def grabProfiles(urls):
    for url in urls:

        browser.visit(url)

        name = browser.find_by_css("span[dir='ltr']")[0].text

        followers = browser.find_by_css("div[class='org-top-card-summary__info-item org-top-card-summary__follower-count']").text
        followers = re.sub("[^\d]", "", followers) # returns only digits
        followers = int(followers)

        employees_on_linkedin = browser.find_by_css("a[data-control-name='topcard_see_all_employees']").text
        employees_on_linkedin = re.sub("[^\d]", "", employees_on_linkedin) # returns only digits
        employees_on_linkedin = int(employees_on_linkedin)

        size = employees_on_linkedin

        url = url

        print(name, followers, employees_on_linkedin)

        linkedin = LinkedIn()
        linkedin.connect()
        linkedin.createTableProfile()
        linkedin.intoProfile(name, followers, employees_on_linkedin, size, url)
        linkedin.select()


In [ ]:
def filterToMostRecent():
    button = browser.find_by_css("li-icon[class='sort-dropdown__icon']")[0]
    button.click()
    button = browser.find_by_css('button[class="flex-grow-1 justify-flex-start ph4 pv2 artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--tertiary ember-view"]')
    button.click()

In [ ]:
def scroll_down():
    
    _data = browser.find_by_css("div[class='occludable-update ember-view']")

    
    while len(_data) > 1:
        _data2 = browser.find_by_css("div[class='occludable-update ember-view']")
        browser.execute_script("window.scrollTo(400, document.body.scrollHeight);")
        _data = browser.find_by_css("div[class='occludable-update ember-view']")
        if (len(_data) == len(_data2)):
            timer = time.time()
            one_minutes_elapsed = timer + 60
            while time.time() < one_minutes_elapsed:
                _data3 = browser.find_by_css("div[class='occludable-update ember-view']")
                browser.execute_script("window.scrollTo(400, document.body.scrollHeight);")
                _data4 = browser.find_by_css("div[class='occludable-update ember-view']")
            if (len(_data2) == len(_data3)):
                print("finished")
                break
    print(len(_data))
        

In [ ]:
def postData(id):
    _data = browser.find_by_css("div[class='occludable-update ember-view']")
    for i in range(len(_data)):
        date = browser.find_by_css("div[class='feed-shared-text-view white-space-pre-wrap break-words ember-view']")[i].text
        date = re.sub("\s+.*", "", date)
        reaction = browser.find_by_css("ul[class='social-details-social-counts ember-view']")[i].text
        print(date)
        try:
            likes = reaction.split("\n")[0].split()[0]
            likes = re.sub("[^\d]", "", likes) # returns only digitS
            int(likes)
        except:
            likes = "0"
            
        try:
            comments = reaction.split("\n")[1].split()[0]
            comments = re.sub("[^\d]", "", comments) # returns only digits
            int(comments)
        except:
            comments = "0"
            
        print("likes: " + str(likes))
        print("comments: " + str(comments))
        
        linkedin = LinkedIn()
        linkedin.connect()
        linkedin.createTablePosts()
            
        linkedin.intoPosts(likes, comments, date, id + 1)
        linkedin.select()
    

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.driver.set_window_size(1280, 720)

In [ ]:
login()

In [ ]:
grabProfiles(urls)

In [ ]:
for id in range(len(urls)):
    browser.visit(urls[id])
    filterToMostRecent()
    scroll_down()
    postData(id)